In [14]:
import re
from json import dump
import requests

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 31) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])


# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]

    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example.json', 'w') as f:
    dump(property_metadata, f)

In [34]:
import json
import pandas as pd
  
# Opening JSON file
f = open('../data/raw/example.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
df2 = pd.DataFrame.from_dict(data, orient="index")
df2

,name,cost_text,coordinates,rooms,desc
https://www.domain.com.au/upstairs-2c-staley-street-brunswick-vic-3056-13168913,Upstairs 2C Staley Street Brunswick VIC 3056,"$35,000 Annually","[-37.7655919, 144.9633048]","[[0 Beds], [0 Baths], [2 Parking]]",1st floor offices/studios\nEasy walking distan...
https://www.domain.com.au/level-3-302-13-15-lake-street-caroline-springs-vic-3023-15994395,"Level 3, 302/13-15 Lake Street Caroline Spring...",4125000 pw,"[-37.7316459, 144.7446886]",[],- Centrally located with tranquil views.\n- Le...
https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767,3502/14-16 The Esplanade St Kilda VIC 3182,"$6,000","[-37.8650177, 144.9746821]","[[3 Beds], [3 Baths], [3 Parking]]",Inspired by the interplay of timeless design a...
https://www.domain.com.au/9-lansdowne-street-blairgowrie-vic-3942-12127675,9 Lansdowne Street Blairgowrie VIC 3942,"$5,000 per week","[-38.372703, 144.7856897]","[[3 Beds], [2 Baths], [3 Parking], []]",Phone enquiry code for this property : 2751
https://www.domain.com.au/52-black-st-brighton-vic-3186-15410646,52 Black St Brighton VIC 3186,From $5500 per week,"[-37.9159452, 144.9989003]","[[3 Beds], [4 Baths], [3 Parking], []]","class=""css-dxogle"">* Unverified feature<svg a..."
...,...,...,...,...,...
https://www.domain.com.au/801-12-queens-road-melbourne-vic-3004-16033913,801/12 Queens Road Melbourne VIC 3004,$900.00,"[-37.8384194, 144.9747126]","[[3 Beds], [2 Baths], [2 Parking]]",Stephanie Lindner
https://www.domain.com.au/6016-228-la-trobe-street-melbourne-vic-3000-15287763,6016/228 La Trobe Street Melbourne VIC 3000,$900/w Furnished,"[-37.8097612, 144.962455]","[[2 Beds], [2 Baths], []]",Positioned on the 60th floor of the prestigiou...
https://www.domain.com.au/7305-228-la-trobe-street-melbourne-vic-3000-15783078,7305/228 La Trobe Street Melbourne VIC 3000,$900/W Unfurnished,"[-37.8097514, 144.9626141]","[[2 Beds], [2 Baths], []]","class=""css-dxogle"">* Unverified feature<svg a..."
https://www.domain.com.au/4916-228-la-trobe-street-melbourne-vic-3000-15888900,4916/228 La Trobe Street Melbourne VIC 3000,$900/W Furnished,"[-37.8097514, 144.9626141]","[[2 Beds], [2 Baths], []]",Positioned on the 4Xth floor of the prestigiou...


In [135]:
l = []
for i in range(len(df2)):
    if(len(df2['rooms'][i]) != 0):
        l.append(True)
    else:
        l.append(False)
df3 = df2[l]

In [131]:
import re
bedroom_list = []
bathroom_list = []
parking_list = []
for i in range(len(df3)):
    bedroom_list.append(re.search('\d+', df3['rooms'][i][0][0]).group())
    bathroom_list.append(re.search('\d+', df3['rooms'][i][1][0]).group())
    if(len(df3['rooms'][i][2])!= 0):
        parking_list.append(re.search('\d+', df3['rooms'][i][2][0]).group())
    else:
        parking_list.append('0')
    # df3['bathroom'][i] = re.search('\d+', df3['rooms'][i][1][0]).group()
    # df3['parking'][i] = re.search('\d+', df3['rooms'][i][2][0]).group()

In [136]:
df3['bedroom'] = bedroom_list
df3['bathroom'] = bathroom_list
df3['parking'] = parking_list

/var/folders/rx/_11wjxvj4hx57z74nvgkbvpr0000gn/T/ipykernel_2710/850092725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bedroom'] = bedroom_list
/var/folders/rx/_11wjxvj4hx57z74nvgkbvpr0000gn/T/ipykernel_2710/850092725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bathroom'] = bathroom_list
/var/folders/rx/_11wjxvj4hx57z74nvgkbvpr0000gn/T/ipykernel_2710/850092725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [137]:
df3

,name,cost_text,coordinates,rooms,desc,bedroom,bathroom,parking
https://www.domain.com.au/upstairs-2c-staley-street-brunswick-vic-3056-13168913,Upstairs 2C Staley Street Brunswick VIC 3056,"$35,000 Annually","[-37.7655919, 144.9633048]","[[0 Beds], [0 Baths], [2 Parking]]",1st floor offices/studios\nEasy walking distan...,0,0,2
https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767,3502/14-16 The Esplanade St Kilda VIC 3182,"$6,000","[-37.8650177, 144.9746821]","[[3 Beds], [3 Baths], [3 Parking]]",Inspired by the interplay of timeless design a...,3,3,3
https://www.domain.com.au/9-lansdowne-street-blairgowrie-vic-3942-12127675,9 Lansdowne Street Blairgowrie VIC 3942,"$5,000 per week","[-38.372703, 144.7856897]","[[3 Beds], [2 Baths], [3 Parking], []]",Phone enquiry code for this property : 2751,3,2,3
https://www.domain.com.au/52-black-st-brighton-vic-3186-15410646,52 Black St Brighton VIC 3186,From $5500 per week,"[-37.9159452, 144.9989003]","[[3 Beds], [4 Baths], [3 Parking], []]","class=""css-dxogle"">* Unverified feature<svg a...",3,4,3
https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303,4203/35 Spring Street Melbourne VIC 3000,$4500 Per Week,"[-37.8141725, 144.9740049]","[[3 Beds], [2 Baths], [4 Parking]]",Possibly the most prestigious CBD address in t...,3,2,4
...,...,...,...,...,...,...,...,...
https://www.domain.com.au/801-12-queens-road-melbourne-vic-3004-16033913,801/12 Queens Road Melbourne VIC 3004,$900.00,"[-37.8384194, 144.9747126]","[[3 Beds], [2 Baths], [2 Parking]]",Stephanie Lindner,3,2,2
https://www.domain.com.au/6016-228-la-trobe-street-melbourne-vic-3000-15287763,6016/228 La Trobe Street Melbourne VIC 3000,$900/w Furnished,"[-37.8097612, 144.962455]","[[2 Beds], [2 Baths], []]",Positioned on the 60th floor of the prestigiou...,2,2,0
https://www.domain.com.au/7305-228-la-trobe-street-melbourne-vic-3000-15783078,7305/228 La Trobe Street Melbourne VIC 3000,$900/W Unfurnished,"[-37.8097514, 144.9626141]","[[2 Beds], [2 Baths], []]","class=""css-dxogle"">* Unverified feature<svg a...",2,2,0
https://www.domain.com.au/4916-228-la-trobe-street-melbourne-vic-3000-15888900,4916/228 La Trobe Street Melbourne VIC 3000,$900/W Furnished,"[-37.8097514, 144.9626141]","[[2 Beds], [2 Baths], []]",Positioned on the 4Xth floor of the prestigiou...,2,2,0
